<a href="https://colab.research.google.com/github/10100111/Display-of-HW1/blob/main/111_Light_Web_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задание Lite

Для пользователя https://www.kinopoisk.ru/user/1127711/

Выгрузите все оценки к фильмам на первой странице и сохраните в pandas dataFrame

 Разбор: https://colab.research.google.com/drive/1JaQkksMnBz2aV7o0PX-FLpdoKkkWK7K0?usp=sharing

# ДЗ

In [8]:
# Библиотеки
from bs4 import BeautifulSoup
import pandas as pd
import requests


In [4]:
# Для обхода Капчи добавим Header из браузера
# -> из консоли разработчика по правой кнопке мыши -> посмотреть код -> откроется справа
# -> Network
# -> /votes -> user agent - в нем все о браузере, этот словарь копи-пастим в header
header = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
'accept-encoding': 'gzip, deflate, br',
'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
'cache-control': 'max-age=0',
'refeheaderrer': 'https://www.themoviedb.org/',
'upgrade-insecure-requests': '1',
'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.106 Safari/537.36'
}

In [5]:
# Попробуем получить все оценки пользователя
user_id = 1127711

# для первой страницы - /list/ord/date/page/1/#list - это первая страничка листа  оценками
# %d - user_id подставится в ссылку
url = 'https://www.kinopoisk.ru/user/%d/votes/list/ord/date/page/1/#list' % (user_id)

# Чтобы получить доступ к контенту запишем результат в переменную r
r = requests.get(url, headers=header)
# Возвращаем контент
r.content


b'\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" xmlns:og="http://opengraphprotocol.org/schema/" dir="ltr" lang="ru" prefix="og: http://ogp.me/ns# video: http://ogp.me/ns/video# ya: http://webmaster.yandex.ru/vocabularies/">\n<head profile="http://gmpg.org/xfn/11">\n    <title>\xd0\x9f\xd1\x80\xd0\xbe\xd1\x84\xd0\xb8\xd0\xbb\xd1\x8c: maksimkat1 - \xd0\x9e\xd1\x86\xd0\xb5\xd0\xbd\xd0\xba\xd0\xb8</title>\n\n    <meta http-equiv="content-type" content="text/html; charset=utf-8" />\n    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />\n    <meta property="fb:app_id" content="121953784483000"/>\n    <meta name="application-name" content="\xd0\x9a\xd0\xb8\xd0\xbd\xd0\xbe\xd0\xbf\xd0\xbe\xd0\xb8\xd1\x81\xd0\xba">\n    <meta content="width=960" name="viewport" />\n    <meta property="fb:pages" content="152308956519" />\n\n    <link rel="search" type="applicat

In [6]:
# Пример HTML-кода для film_list (для наглядности)
film_list_html = '''
<div class="profileFilmsList">
    <div class="item">
        <div class="NameRus">Фильм 1</div>
        <div class="vote">8.5</div>
    </div>
    <div class="item_even">
        <div class="NameRus">Фильм 2</div>
        <div class="vote">7.2</div>
    </div>
    <!-- Добавьте остальные фильмы здесь -->
</div>
'''
# Beautiful Soup
# Получаем объект soup, который является объектом BeautifulSoup'a
soup = BeautifulSoup(r.text) # конвертирует скрауленый текст в строку
# Если надо найти инфо, которая хранится в определенном теге
film_list = soup.find('div', {'class': 'profileFilmsList'})

# Некоторые фильмы лежат в классе item, а некоторые в классе item_even
#!!! find_all() найдет все, что соответствует шаблону и вернет в виде списка
# !!! find() только первый попавшийся варинт
item = film_list.find_all('div', {'class': ['item', 'item_even']})

# Создаем объект soup из film_list_html (закомментировано для теста)
soup = BeautifulSoup(film_list_html, 'html.parser')

# Создаем списки для хранения данных
film_names = []
user_votes = []

# Итерируемся по фильмам и извлекаем данные
for film in item:
    film_name_elem = film.find('div', {'class': 'nameRus'})
    user_vote_elem = film.find('div', {'class': 'vote'})

    if film_name_elem is not None and user_vote_elem is not None:
        film_name = film_name_elem.text
        user_vote = float(user_vote_elem.text)
        film_names.append(film_name)
        user_votes.append(user_vote)

# Создаем датафрейм из списков
df = pd.DataFrame({'film_name': film_names, 'user_vote': user_votes})

# Выводим полученный датафрейм
print(df)


                                     film_name  user_vote
0                             Мизантроп (2022)        6.0
1                       Счастье (сериал, 2021)        5.0
2     Декстер: Новая кровь (мини-сериал, 2021)        9.0
3                              Выжившая (2017)        7.0
4                           Многоэтажка (2022)       10.0
5                    Жизнь Дэвида Гейла (2002)        9.0
6                        На твоей волне (2019)        9.0
7                                Другие (2001)        8.0
8                           Милые кости (2009)        9.0
9                       Последний рубеж (2013)        4.0
10                           Погружение (2017)        7.0
11                         Демон внутри (2016)       10.0
12                                Кукла (2015)        8.0
13                       Начало (сериал, 2022)       10.0
14                         Исходный код (2011)       10.0
15        Необычайные приключения Адель (2010)        5.0
16            

In [7]:
len(item)

50

# Урок. Парсим Кинопоиск с помощью Beautifulsoup

In [ ]:
# Для обхода Капчи добавим Header из браузера
# -> из консоли разработчика по правой кнопке мыши -> посмотреть код -> откроется справа
# -> Network
# -> /votes -> user agent - в нем все о браузере, этот словарь копи-пастим в header
header = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
'accept-encoding': 'gzip, deflate, br',
'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
'cache-control': 'max-age=0',
'refeheaderrer': 'https://www.themoviedb.org/',
'upgrade-insecure-requests': '1',
'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.106 Safari/537.36'
}

In [ ]:
# Попробуем получить все оценки пользователя
import requests
user_id = 1127711

# для первой страницы - /list/ord/date/page/1/#list - это первая страничка листа  оценками
# %d - user_id подставится в ссылку
url = 'https://www.kinopoisk.ru/user/%d/votes/list/ord/date/page/1/#list' % (user_id)

# Чтобы получить доступ к контенту запишем результат в переменную r
r = requests.get(url, headers=header)
# Возвращаем контент
r.content


In [ ]:
# Сохраним данные в файл и откроем
with open('test2.html', 'wb') as output_file:
  output_file.write(r.text.encode('utf-8'))

In [ ]:
from bs4 import BeautifulSoup
from lxml import html

# Beautiful Soup
# Получаем объект soup, который является объектом BeautifulSoup'a
soup = BeautifulSoup(r.text) # конвертирует скрауленый текст в строку
# Если надо найти инфо, которая хранится в определенном теге
film_list = soup.find('div', {'class': 'profileFilmsList'})

# То же самое можно сделать с помощью библиотеки XPATH
# lxml
tree = html.fromstring(r.text)
film_list_lxml = tree.xpath('//div[@class = "profileFilmsList"]')[0]

In [ ]:
film_list

<div class="profileFilmsList">
<div class="top">
<div class="num">№</div>
<div class="name">фильм</div>
<div class="date">дата и время</div>
<div class="ya-sync"></div>
<div class="vote">оценка maksimkat1</div>
<div class="vote myvote">моя<br/>оценка</div>
<div class="clear"></div>
</div>
<div class="item">
<div class="num">1</div>
<div class="info">
<div class="nameRus"><a href="/film/1263902/">Мизантроп (2022)</a></div>
<div class="nameEng">To Catch A Killer</div>
<div class="rating">
<b>7.131</b>
<span class="text-grey">(175 267)</span>
<span class="text-grey">119 мин.</span>
                                              
                                         </div>
</div>
<div class="date">16.07.2023, 01:50</div>
<div class="vote">6</div>
<div class="selects vote_widget">
<span id="rating_user_1263902" title="поставить оценку"> </span>
<div class="MyKP_Folder_Select shortestselect MyKP_Folder_1263902" mid="1263902" type="film"><s class="dot"></s><div class="arrow"></div></div>
<

In [ ]:
# Некоторые фильмы лежат в классе item, а некоторые в классе item_even
#!!! find_all() найдет все, что соответствует шаблону и вернет в виде списка
# !!! find() только первый попавшийся варинт
item = film_list.find_all('div', {'class': ['item', 'item_even']})[0]
item_lxml = film_list_lxml.xpath('//div[@class = "item"]')[0]

# item = film_list.find_all('div', {'class': ['item', 'item_even']})

In [ ]:
item

<div class="item">
<div class="num">1</div>
<div class="info">
<div class="nameRus"><a href="/film/1263902/">Мизантроп (2022)</a></div>
<div class="nameEng">To Catch A Killer</div>
<div class="rating">
<b>7.131</b>
<span class="text-grey">(175 238)</span>
<span class="text-grey">119 мин.</span>
                                              
                                         </div>
</div>
<div class="date">16.07.2023, 01:50</div>
<div class="vote">6</div>
<div class="selects vote_widget">
<span id="rating_user_1263902" title="поставить оценку"> </span>
<div class="MyKP_Folder_Select shortestselect MyKP_Folder_1263902" mid="1263902" type="film"><s class="dot"></s><div class="arrow"></div></div>
<script nonce="">
                                        $(".MyKP_Folder_1263902").folder({objId: 1263902, objType: "film", template : "shortest"});
                                        </script>
</div>
<div class="clear"></div>
<script nonce="">
                                      ur

In [ ]:
len(item)

15

In [ ]:
# Если надо что-то найти внутри item, то внутри item вызываем метод find()
# Опять указываем его тег и класс, если в классе еще внутри есть тег + класс, можно снова вызвать метод find()
# Beatiful Soup
movie_link = item.find('div', {'class': 'nameRus'}).find('a').get('href')
movie_desc = item.find('div', {'class': 'nameEng'}).text
print(movie_link, movie_desc)

# # lxml
movie_link = item_lxml.xpath('.//div[@class = "nameRus"]/a/@href')[0]
movie_desc = item_lxml.xpath('.//div[@class = "nameEng"]/text()')[0]
print(movie_link, movie_desc)

/film/1263902/ To Catch A Killer
/film/1263902/ To Catch A Killer
